In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


In [ ]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# === CONFIGURATION ===
USERNAME = "dgause"
PASSWORD = "wT7Ei2zdce!L2kQ"
DOWNLOAD_URL = "https://download.geoservice.dlr.de/ENMAP/files/L2A/2023/07/09/DT0000028166/02/ENMAP01-____L2A-DT0000028166_20230709T064108Z_002_V010402_20240724T202501Z-SPECTRAL_IMAGE_COG.TIF"
OUTPUT_FILE = "./downloads/enmap_image.tif"


# === SETUP HEADLESS CHROME ===
chrome_options = Options()
chrome_options.add_argument("--headless")  # Remove this if you want to watch the browser
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Launch browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# === STEP 1: Navigate to CAS Login Page ===
driver.get(f"https://sso.eoc.dlr.de/eoc/auth/login?service={DOWNLOAD_URL}")
time.sleep(2)

# === STEP 2: Log in ===
driver.find_element(By.NAME, "username").send_keys(USERNAME)
driver.find_element(By.NAME, "password").send_keys(PASSWORD)
driver.find_element(By.NAME, "login").click()
time.sleep(5)  # Wait for login and redirect

# === STEP 3: Extract Cookies from Browser ===
cookies = driver.get_cookies()
driver.quit()
cookie_dict = {cookie['name']: cookie['value'] for cookie in cookies}

# === STEP 4: Use cookies with requests to download the .TIF ===
r = requests.get(DOWNLOAD_URL, cookies=cookie_dict, allow_redirects=True)

# === STEP 5: Save File ===
if r.status_code == 200 and not r.text.startswith("<!DOCTYPE html>"):
    with open(OUTPUT_FILE, "wb") as f:
        f.write(r.content)
    print(f"✅ Downloaded to {OUTPUT_FILE}")
else:
    print(f"❌ Failed with status code {r.status_code}")
    print(r.text[:300])


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="login"]"}
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104786668 cxxbridge1$str$ptr + 2723108
1   chromedriver                        0x000000010477e8dc cxxbridge1$str$ptr + 2690968
2   chromedriver                        0x00000001042d2714 cxxbridge1$string$len + 90428
3   chromedriver                        0x00000001043197c0 cxxbridge1$string$len + 381416
4   chromedriver                        0x000000010435ade8 cxxbridge1$string$len + 649232
5   chromedriver                        0x000000010430d9c8 cxxbridge1$string$len + 332784
6   chromedriver                        0x000000010474a28c cxxbridge1$str$ptr + 2476360
7   chromedriver                        0x0000000

In [15]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# === CONFIGURATION ===
USERNAME = "dgause"
PASSWORD = "wT7Ei2zdce!L2kQ"
DOWNLOAD_URL = "https://download.geoservice.dlr.de/ENMAP/files/L2A/2023/07/09/DT0000028166/02/ENMAP01-____L2A-DT0000028166_20230709T064108Z_002_V010402_20240724T202501Z-SPECTRAL_IMAGE_COG.TIF"
OUTPUT_FILE = "./downloads/enmap_image.tif"

# === SETUP HEADLESS CHROME ===
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Remove comment for headless
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    # STEP 1: Open login page
    login_url = f"https://sso.eoc.dlr.de/eoc/auth/login?service={DOWNLOAD_URL}"
    driver.get(login_url)

    # STEP 2: Login form
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username")))
    driver.find_element(By.ID, "username").send_keys(USERNAME)
    pw_box = driver.find_element(By.ID, "password")
    pw_box.send_keys(PASSWORD)
    pw_box.send_keys(Keys.RETURN)

    # STEP 3: Wait for cookie to be set (not redirect necessarily)
    time.sleep(5)

    # Grab all cookies
    cookies = driver.get_cookies()
    cookie_dict = {cookie['name']: cookie['value'] for cookie in cookies}
    print("✅ Cookies retrieved:", cookie_dict.keys())

finally:
    driver.quit()

# STEP 4: Use session to download file
with requests.Session() as session:
    session.cookies.update(cookie_dict)
    r = session.get(DOWNLOAD_URL, allow_redirects=True)

    if r.status_code == 200 and not r.text.lower().startswith("<!doctype html>"):
        with open(OUTPUT_FILE, "wb") as f:
            f.write(r.content)
        print(f"✅ Download complete: {OUTPUT_FILE}")
    else:
        print(f"❌ Download failed. Status {r.status_code}")
        print(r.text[:500])

✅ Cookies retrieved: dict_keys(['org.springframework.web.servlet.i18n.CookieLocaleResolver.LOCALE', 'TGC'])
❌ Download failed. Status 200
<!DOCTYPE html><html lang="en_US">

<head>
    <meta charset="UTF-8" /><meta http-equiv="X-UA-Compatible" content="IE=edge" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" /><title>EOC UMS: EnMAP Access Service Login</title>
    
        <link rel="stylesheet" type="text/css" href="/eoc/auth/webjars/normalize.css/8.0.1/normalize.css" /><link rel="stylesheet" type="text/css" href="/eoc/auth/webjars/bootstrap/5.3.2/css/bootstrap-grid.min.css" /><link rel="st


In [18]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# === CONFIGURATION ===
USERNAME = "dgause"
PASSWORD = "wT7Ei2zdce!L2kQ"
DOWNLOAD_URL = "https://download.geoservice.dlr.de/ENMAP/files/L2A/2023/07/09/DT0000028166/02/ENMAP01-____L2A-DT0000028166_20230709T064108Z_002_V010402_20240724T202501Z-SPECTRAL_IMAGE_COG.TIF"
OUTPUT_FILE = "./downloads/enmap_image.tif"

# === SETUP HEADLESS CHROME ===
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Enable this for silent run
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    # STEP 1: Open login page
    login_url = f"https://sso.eoc.dlr.de/eoc/auth/login?service={DOWNLOAD_URL}"
    driver.get(login_url)

    # STEP 2: Login form
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username")))
    driver.find_element(By.ID, "username").send_keys(USERNAME)
    pw_box = driver.find_element(By.ID, "password")
    pw_box.send_keys(PASSWORD)
    pw_box.send_keys(Keys.RETURN)

    # STEP 3: Wait briefly after login
    time.sleep(5)

    # STEP 4: Grab cookies from Chrome
    cookies = driver.get_cookies()
    cookie_dict = {cookie['name']: cookie['value'] for cookie in cookies}
    print("✅ Cookies retrieved:", cookie_dict.keys())

    # STEP 5: Use requests to download with authenticated cookies
    print("📥 Starting file download with session...")
    with requests.Session() as session:
        session.cookies.update(cookie_dict)
        r = session.get(DOWNLOAD_URL, stream=True)

        if r.status_code == 200 and not r.text.lower().startswith("<!doctype html>"):
            with open(OUTPUT_FILE, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            print(f"✅ Download complete: {OUTPUT_FILE}")
        else:
            print(f"❌ Download failed. Status: {r.status_code}")
            print(r.text[:500])

finally:
    driver.quit()


✅ Cookies retrieved: dict_keys(['org.springframework.web.servlet.i18n.CookieLocaleResolver.LOCALE', 'TGC'])
📥 Starting file download with session...
❌ Download failed. Status: 200
<!DOCTYPE html><html lang="en_US">

<head>
    <meta charset="UTF-8" /><meta http-equiv="X-UA-Compatible" content="IE=edge" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" /><title>EOC UMS: EnMAP Access Service Login</title>
    
        <link rel="stylesheet" type="text/css" href="/eoc/auth/webjars/normalize.css/8.0.1/normalize.css" /><link rel="stylesheet" type="text/css" href="/eoc/auth/webjars/bootstrap/5.3.2/css/bootstrap-grid.min.css" /><link rel="st


In [21]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# === CONFIGURATION ===
USERNAME = "dgause"
PASSWORD = "wT7Ei2zdce!L2kQ"
DOWNLOAD_URL = "https://download.geoservice.dlr.de/ENMAP/files/L2A/2023/07/09/DT0000028166/02/ENMAP01-____L2A-DT0000028166_20230709T064108Z_002_V010402_20240724T202501Z-SPECTRAL_IMAGE_COG.TIF"
DOWNLOAD_DIR = "./downloads"

# === CHROME OPTIONS TO ENABLE DOWNLOAD ===
chrome_options = Options()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)

# === START BROWSER ===
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    # STEP 1: Go to login URL
    login_url = f"https://sso.eoc.dlr.de/eoc/auth/login?service={DOWNLOAD_URL}"
    driver.get(login_url)

    # STEP 2: Log in
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "username")))
    driver.find_element(By.ID, "username").send_keys(USERNAME)
    pw_box = driver.find_element(By.ID, "password")
    pw_box.send_keys(PASSWORD)
    pw_box.send_keys(Keys.RETURN)

    # STEP 3: Wait for file to download
    print("✅ Login successful. Waiting for download to start...")
    time.sleep(15)

    # Wait for file to appear in download dir
    downloaded_file = os.path.join(DOWNLOAD_DIR, os.path.basename(DOWNLOAD_URL))
    print(f"📥 Looking for: {downloaded_file}")
    while not os.path.exists(downloaded_file):
        print("⏳ Waiting for download to complete...")
        time.sleep(5)

    print(f"✅ Download complete: {downloaded_file}")

finally:
    driver.quit()


✅ Login successful. Waiting for download to start...
📥 Looking for: ./downloads/ENMAP01-____L2A-DT0000028166_20230709T064108Z_002_V010402_20240724T202501Z-SPECTRAL_IMAGE_COG.TIF
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download to complete...
⏳ Waiting for download t

KeyboardInterrupt: 

In [23]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# === CONFIG ===
USERNAME = "dgause"
PASSWORD = "wT7Ei2zdce!L2kQ"
DOWNLOAD_URL = "https://download.geoservice.dlr.de/ENMAP/files/L2A/2023/07/09/DT0000028166/02/ENMAP01-____L2A-DT0000028166_20230709T064108Z_002_V010402_20240724T202501Z-SPECTRAL_IMAGE_COG.TIF"
DOWNLOAD_DIR = os.path.abspath("downloads")
FILENAME = os.path.basename(DOWNLOAD_URL)
FINAL_PATH = os.path.join(DOWNLOAD_DIR, FILENAME)
CRDOWNLOAD_PATH = FINAL_PATH + ".crdownload"

# === SETUP CHROME ===
chrome_options = Options()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    login_url = f"https://sso.eoc.dlr.de/eoc/auth/login?service={DOWNLOAD_URL}"
    driver.get(login_url)

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "username")))
    driver.find_element(By.ID, "username").send_keys(USERNAME)
    password_input = driver.find_element(By.ID, "password")
    password_input.send_keys(PASSWORD)
    password_input.send_keys(Keys.RETURN)

    print("✅ Login successful. Waiting for download to complete...")

    # === Wait for .crdownload to appear and disappear ===
    while not os.path.exists(FINAL_PATH) and not os.path.exists(CRDOWNLOAD_PATH):
        print("⏳ Waiting for download to start...")
        time.sleep(2)

    while os.path.exists(CRDOWNLOAD_PATH):
        print("⏳ Download in progress...")
        time.sleep(3)

    print(f"✅ Download complete: {FINAL_PATH}")

finally:
    driver.quit()
    print("✅ Selenium driver closed.")


✅ Login successful. Waiting for download to complete...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...
⏳ Waiting for download to start...

KeyboardInterrupt: 

In [30]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# === CONFIG ===
USERNAME = "dgause"
PASSWORD = "wT7Ei2zdce!L2kQ"
DOWNLOAD_URL = "https://download.geoservice.dlr.de/ENMAP/files/L2A/2023/07/09/DT0000028166/02/ENMAP01-____L2A-DT0000028166_20230709T064108Z_002_V010402_20240724T202501Z-SPECTRAL_IMAGE_COG.TIF"
FILENAME = os.path.basename(DOWNLOAD_URL)
DOWNLOAD_DIR = os.path.join(os.getcwd(), "downloads")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

FINAL_PATH = os.path.join(DOWNLOAD_DIR, FILENAME)
CRDOWNLOAD_PATH = FINAL_PATH + ".crdownload"

# === SETUP CHROME ===
chrome_options = Options()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    login_url = f"https://sso.eoc.dlr.de/eoc/auth/login?service={DOWNLOAD_URL}"
    driver.get(login_url)

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "username")))
    driver.find_element(By.ID, "username").send_keys(USERNAME)
    password_input = driver.find_element(By.ID, "password")
    password_input.send_keys(PASSWORD)
    password_input.send_keys(Keys.RETURN)

    print("✅ Login successful. Waiting for download to start...")

    import glob

    # === Wait for any .crdownload file to appear ===
    print("👀 Watching for downloads to begin...")
    download_started = False
    while True:
        cr_files = glob.glob(os.path.join(DOWNLOAD_DIR, "*.crdownload"))
        if cr_files:
            download_started = True
            print("⏳ Download started...")
            break
        time.sleep(1)

    # === Wait for all .crdownload files to disappear ===
    print("⏳ Waiting for download to complete...")
    while glob.glob(os.path.join(DOWNLOAD_DIR, "*.crdownload")):
        time.sleep(2)

    print("✅ Download complete.")

finally:
    driver.quit()
    print("✅ Selenium driver closed.")


✅ Login successful. Waiting for download to start...
👀 Watching for downloads to begin...
⏳ Download started...
⏳ Waiting for download to complete...
✅ Download complete.
✅ Selenium driver closed.
